In [1]:
from src.elastic.indexer import load_documents

In [2]:
documents_file = "data/input/faqs/documents.json"

In [ ]:
documentos = load_documents(documents_file)
documentos[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp',
 'id': 'c02e79ef'}

In [4]:
count_dict = {}
for doc in documentos:
    count_dict[doc["id"]] = count_dict.get(doc["id"], 0) + 1
    
for key, value in count_dict.items():
    if value > 1:
        print(key, value)


593f7569 2


In [5]:
for doc in documentos:
    if doc["id"] == "593f7569":
        print(doc)
    else:
        continue

{'text': "They both do the same, it's just less typing from the script.\nAsked by Andrew Katoch, Added by Edidiong Esu", 'section': '6. Decision Trees and Ensemble Learning', 'question': 'Does it matter if we let the Python file create the server or if we run gunicorn directly?', 'course': 'machine-learning-zoomcamp', 'id': '593f7569'}
{'text': "They both do the same, it's just less typing from the script.", 'section': '6. Decision Trees and Ensemble Learning', 'question': 'Does it matter if we let the Python file create the server or if we run gunicorn directly?', 'course': 'machine-learning-zoomcamp', 'id': '593f7569'}
